<a href="https://colab.research.google.com/github/thomas774/Python/blob/master/IRAQ_NUTELLA_T350x15_nnet_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

******************************* **WARNING : Please do not forget to update the name of the output files for SFA file and for prediction file**





******************************* **WARNING**

*************************************************************************************************************************

In [0]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
# fix random seed for reproducibility
np.random.seed(7)

In [0]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**I) EVALUATE SFA RESULTS**

In [0]:
# Import the data set
Train = pd.read_csv("/content/gdrive/My Drive/ML FERRERO/Iraq/Nutella/T350x15/Test past Sales Forecast Accuracy/Train/IRAQ_NUTELLA_T350x15.csv")
Test = pd.read_csv("/content/gdrive/My Drive/ML FERRERO/Iraq/Nutella/T350x15/Test past Sales Forecast Accuracy/Test/IRAQ_NUTELLA_T350x15.csv")

Train_X, Train_Y = Train.iloc[:,4:6], Train.iloc[:,8:9]
Test_X, Test_Y = Test.iloc[:,4:6], Test.iloc[:,8:9]

In [0]:
# create model
model = Sequential()
model.add(Dense(8, input_dim=2, activation='linear'))
#model.add(Dense(4, activation='linear'))
model.add(Dense(1, activation='linear'))

In [0]:
from keras import optimizers

# Loss function
#def root_mean_squared_error(y_true, y_pred):
#        return np.sqrt(np.mean(np.square(y_pred - y_true), axis=-1))

# Optimizer
sgd = optimizers.SGD(lr=0.0000001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mae'])

In [0]:
# Fit the model
model.fit(Train_X, Train_Y, epochs=150, batch_size=10, verbose=0)

In [0]:
# evaluate the model
scores = model.evaluate(Test_X, Test_Y)
print("\n%s: %.2f" % (model.metrics_names[1], scores[1]))

3/3 [==============================] - 0s 6ms/step

mean_absolute_error: 139.74


In [0]:
# calculate predictions
predictions = model.predict(Test_X)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(predictions)

[[430.42914]
 [440.49188]
 [449.0775 ]]


In [0]:
# Calculate SFA
Result = pd.DataFrame(predictions)
Result['ini'] = Test_Y
SFA = 1-(abs(sum(Result[0])-sum(Result['ini']))/sum(Result['ini']))
SFA*100, sum(Result[0]), sum(Result['ini'])

(53.46193670622698, 1319.99853515625, 900.7888499999999)

In [0]:
# Create the output file
Array = []
Array.append(sum(Result[0]))
Array.append(sum(Result['ini']))

Output = pd.DataFrame(Array)
Output.to_csv("/content/gdrive/My Drive/ML FERRERO/Iraq/Nutella/T350x15/SFA_201848.csv", sep=',', encoding='utf-8')

**II) Create predictions for the 3 next weeks and train the model on complete data**

In [0]:
# Create predictions for the 3 next weeks
Train_pred = pd.read_csv("/content/gdrive/My Drive/ML FERRERO/Iraq/Nutella/T350x15/Predictions/Train/IRAQ_NUTELLA_T350x15.csv")
Test_pred = pd.read_csv("/content/gdrive/My Drive/ML FERRERO/Iraq/Nutella/T350x15/Predictions/Test/IRAQ_NUTELLA_T350x15.csv")

Train_pred_X, Train_pred_Y = Train_pred.iloc[:,4:6], Train_pred.iloc[:,8:9]
Test_pred_X = Test_pred.iloc[:,4:6]

# Fit the model
model.fit(Train_pred_X, Train_pred_Y, epochs=150, batch_size=10, verbose=0)

In [0]:
# evaluate the model
scores_pred = model.evaluate(Train_pred_X, Train_pred_Y)
print("\n%s: %.2f" % (model.metrics_names[1], scores_pred[1]))

120/120 [==============================] - 0s 84us/step

mean_absolute_error: 126.05


In [0]:
# calculate predictions
predictions_3nextweeks = model.predict(Test_pred_X)
# round predictions
rounded_pred = [round(x[0]) for x in predictions_3nextweeks]
print(predictions_3nextweeks)

[[448.96588 ]
 [457.1752  ]
 [ 14.436336]]


In [0]:
# Create the output file
Output_pred = pd.DataFrame(predictions_3nextweeks)
Output_pred['Customer'] = Test_pred['Customer']
Output_pred['Product_Group'] = Test_pred['Product_Group']
Output_pred['Item'] = Test_pred['Item']
Output_pred['Year_Nb'] = Test_pred['Year_Nb']
Output_pred['Month_Nb'] = Test_pred['Month_Nb']
Output_pred['Week_Nb'] = Test_pred['Week_Nb']
Output_pred['Year_Week_Nb'] = Test_pred['Year_Week_Nb']
Output_pred['Current_Week'] = Test_pred['Current_Week']
Output_pred.to_csv("/content/gdrive/My Drive/ML FERRERO/Iraq/Nutella/T350x15/predictions_201851.csv", sep=',', encoding='utf-8')